In [2]:
import random
import numpy as np 


class ReplayBuffer:
    def __init__(self, buffer_size, minibatch_size, observation_size):
        """
        Args:
            size (integer): The size of the replay buffer.              
            minibatch_size (integer): The sample size.
            seed (integer): The seed for the random number generator. 
        """
        self.buffer = []
        self.minibatch_size = minibatch_size
        #random.seed(seed)
        self.max_size = buffer_size
        self.pos = 0
        self.full = False
        self.states = np.zeros((self.max_size,observation_size))
        self.next_states = np.zeros((self.max_size,observation_size))
        self.actions = np.zeros(self.max_size,dtype=np.int8)
        self.rewards = np.zeros(self.max_size)
        self.terminals = np.zeros(self.max_size,dtype=np.int8)


    def append(self, state, action, reward, terminal, next_state):
        """
        Args:
            state (Numpy array): The state.              
            action (integer): The action.
            reward (float): The reward.
            terminal (integer): 1 if the next state is a terminal state and 0 otherwise.
            next_state (Numpy array): The next state.           
        """
        self.states[self.pos] = state
        self.actions[self.pos] = action
        self.rewards[self.pos] = reward
        self.terminals[self.pos] = terminal
        self.next_states[self.pos] = next_state
        self.pos += 1
        if(self.pos==self.max_size):
            self.pos = 0
            self.full = True


    def sample(self):
        """
        Returns:
            A list of transition tuples including state, action, reward, terinal, and next_state
        """
        if(self.full):
            idxs = np.random.randint(0,self.max_size,size=self.minibatch_size) 
        else:
            idxs = np.random.randint(0,self.pos,size=self.minibatch_size)
        sample_ = [self.states[idxs],self.actions[idxs],self.rewards[idxs],self.terminals[idxs],
                   self.next_states[idxs]]
        #print(sample_)
        return sample_

    def size(self):
        if(self.full):
            return self.max_size
        else:
            return self.pos
    
    def reset(self):
        self.full = False
        self.pos = 0

In [3]:
import torch

class ActorCritic(torch.nn.Module):
    def __init__(self,network_config) -> None:
        super(ActorCritic,self).__init__()
        self.state_dim = network_config.get("state_dim")
        self.num_hidden_units = network_config.get("num_hidden_units")
        self.num_actions = network_config.get("num_actions")

        self.layer1 = torch.nn.Linear(self.state_dim, self.num_hidden_units)
        self.layer2 = torch.nn.Linear(self.num_hidden_units,self.num_hidden_units)
        self.policy_layer = torch.nn.Linear(self.num_hidden_units, self.num_actions)
        self.value_layer = torch.nn.Linear(self.num_hidden_units,1)
        self.relu = torch.nn.ReLU()
        self.softmax = torch.nn.Softmax(dim=-1)
        self.actor = torch.nn.Sequential(
            self.layer1,self.relu,self.policy_layer,self.softmax
        )
        self.critic = torch.nn.Sequential(
            self.layer1,self.relu,self.value_layer
        )

    def forward(self, x):
        policy = self.actor(x)
        value = self.critic(x)
        return policy,value

In [4]:
def get_td_error_double_dqn(states, next_states, actions, rewards, discount, terminals, target_network, current_q_network):
    with torch.no_grad():
        # The idea of Double DQN is to get max actions from current network
        # and to get Q values from target_network for next states. 
        q_next_mat = current_q_network(next_states)
        max_actions = torch.argmax(q_next_mat,1)
        double_q_mat = target_network(next_states)
    batch_indices = torch.arange(q_next_mat.shape[0])
    double_q_max = double_q_mat[batch_indices,max_actions]
    target_vec = rewards+discount*double_q_max*(torch.ones_like(terminals)-terminals)
    q_mat = current_q_network(states)
    batch_indices = torch.arange(q_mat.shape[0])
    q_vec = q_mat[batch_indices,actions]
    #delta_vec = target_vec - q_vec
    return target_vec,q_vec

def get_td_error_dqn(states, next_states, actions, rewards, discount, terminals, target_network, current_q_network):
    with torch.no_grad():
        q_next_mat = target_network(next_states)
        q_max = torch.max(q_next_mat,1)[0]
    batch_indices = torch.arange(q_next_mat.shape[0])
    target_vec = rewards+discount*q_max*(torch.ones_like(terminals)-terminals)
    q_mat = current_q_network(states)
    batch_indices = torch.arange(q_mat.shape[0])
    q_vec = q_mat[batch_indices,actions]
    return target_vec,q_vec

In [5]:
def optimize_network(experiences, discount, optimizer, target_network, current_q_network,device,double_dqn=False):
    """
    Args:
        experiences (Numpy array): The batch of experiences including the states, actions,
                                   rewards, terminals, and next_states.
        discount (float): The discount factor.
        network (ActionValueNetwork): The latest state of the network that is getting replay updates.
        current_q (ActionValueNetwork): The fixed network used for computing the targets,
                                        and particularly, the action-values at the next-states.
    """
    # Get states, action, rewards, terminals, and next_states from experiences
    states = experiences[0]
    actions = experiences[1]
    rewards = experiences[2]
    terminals = experiences[3]
    next_states = experiences[4]
    # numpy arrays to tensors
    states = torch.tensor(states,dtype=torch.float32,device=device)
    
    #print(states.shape)
    next_states = torch.tensor(next_states,dtype=torch.float32,device=device)
    rewards = torch.tensor(rewards,dtype=torch.float32,device=device)
    terminals = torch.tensor(terminals,dtype=torch.int,device=device)
    actions = torch.tensor(actions,dtype=torch.int,device=device)
    #map(list, zip(*experiences))
    #states = torch.concatenate(states)
    #next_states = torch.concatenate(next_states)
    #rewards = torch.tensor(rewards,dtype=torch.float32,device=device)
    #terminals = torch.tensor(terminals,dtype=torch.float32,device=device)
    batch_size = states.shape[0]
    # Compute TD error using the get_td_error function
    # Note that q_vec is a 1D array of shape (batch_size)
    if(double_dqn):
        target_vec,q_vec = get_td_error_double_dqn(states, next_states, actions, rewards, discount, terminals, target_network, current_q_network)
    else:
        target_vec,q_vec = get_td_error_dqn(states, next_states, actions, rewards, discount, terminals, target_network, current_q_network)
    loss_fun = torch.nn.MSELoss()
    loss = loss_fun(target_vec,q_vec)
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(target_network.parameters(), 10)
    optimizer.step()
    return loss.detach().cpu().numpy()

In [6]:
import torch
import random
from copy import deepcopy
import numpy as np




class A2CAgent:
    def __init__(self):
        self.name = "A2C"
        torch.autograd.set_detect_anomaly(True)
        self.device = None
        self.seed = 1 # random seed. Later can be changed by using set_seed method

    def set_seed(self,seed=1):
        self.seed = seed
        #random.seed(self.seed)
    

    def set_device(self,device):
        self.device = device
    
    def agent_init(self, agent_config):
        if(self.device==None):
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.state_dim = agent_config["network_config"].get("state_dim")
        self.num_hidden_layers = agent_config["network_config"].get("num_hidden_units")
        self.num_actions = agent_config["network_config"].get("num_actions")
        self.dueling = agent_config["network_config"].get("dueling")
        
        self.actor_critic_network = ActorCritic(agent_config['network_config']).to(self.device)
        
        self.actor_step_size = 1e-4 #agent_config['actor_step_size']
        self.critic_step_size =  1e-3 #agent_config['critic_step_size']
        self.avg_reward_step_size = 1e-3
        self.num_actions = agent_config['network_config']['num_actions']
        self.discount = agent_config['gamma']
        self.time_step = 0
        self.loss = []
        self.episode_rewards = []
        self.loss_capacity = 5_000
        self.last_state = None
        self.last_action = None
        self.sum_rewards = 0
        self.episode_steps = 0
        self.actor_optimizer = torch.optim.Adam(self.actor_critic_network.actor.parameters(),lr=self.actor_step_size)
        self.critic_optimizer = torch.optim.Adam(self.actor_critic_network.critic.parameters(),lr=self.critic_step_size)
        self.avg_reward = 0
        self.I = 1
        self.states = []
        self.actions = []
        self.rewards = []
        self.next_states = []
        self.terminals = []

    def select_action(self,state):
        state = torch.tensor(np.array(state),dtype=torch.float32,device=self.device)
        policy,_ = self.actor_critic_network(state)
        action_probabilities = policy.squeeze().detach().numpy()
        action = np.random.choice(len(action_probabilities), p=action_probabilities)
        return action

    def policy(self,state):
        #state = torch.tensor(state,dtype=torch.float32)
        
        policy,_ = self.actor_critic_network(state)
        m = torch.distributions.Categorical(policy)
        action = m.sample()
        lp = m.log_prob(action)
        #action = torch.multinomial(policy, 1).item()
        return action.item(),lp
    
    def value(self,state):
        #state = torch.tensor(state,dtype=torch.float32)
        _,value = self.actor_critic_network(state)
        return value

    # Work Required: No.
    def agent_start(self, state):
        """The first method called when the experiment starts, called after
        the environment starts.
        Args:
            state (Numpy array): the state from the
                environment's evn_start function.
        Returns:
            The first action the agent takes.
        """
        self.sum_rewards = 0
        self.episode_steps = 0
        self.I = 1
        self.last_state = state #torch.tensor(np.array([state]),dtype=torch.float32,device=self.device)
        self.last_action = self.select_action(state)
        self.time_step += 1
        return self.last_action

    def agent_step(self, reward, state):
        """A step taken by the agent.
        Args:
            reward (float): the reward received for taking the last action taken
            state (Numpy array): the state from the
                environment's step based, where the agent ended up after the
                last step
        Returns:
            The action the agent is taking.
        """
        self.sum_rewards += reward
        self.episode_steps += 1
        action = self.select_action(state)
        self.states.append(self.last_state)
        self.actions.append(self.last_action)
        self.rewards.append(reward)
        self.terminals.append(False)
        self.next_states.append(state)
        self.last_action = action
        self.last_state = state
        return action

    def agent_end(self, reward):
        """Run when the agent terminates.
        Args:
            reward (float): the reward the agent received for entering the
                terminal state.
        """
        self.sum_rewards += reward
        self.episode_steps += 1
        
        self.states.append(self.last_state)
        self.actions.append(self.last_action)
        self.rewards.append(reward)
        state = np.zeros_like(self.last_state)
        self.terminals.append(True)
        self.next_states.append(state)
        loss=self.agent_update()
        return loss

    def discount_rewards(self, rewards):
        # Compute the gamma-discounted rewards over an episode
        
        running_add = 0
        discounted_r = np.zeros_like(rewards)
        for i in reversed(range(0,len(rewards))):
            
            running_add = running_add * self.discount + rewards[i]
            discounted_r[i] = running_add
        discounted_r -= np.mean(discounted_r) # normalizing the result
        discounted_r /= np.std(discounted_r) # divide by standard deviation
        return discounted_r

    def agent_update(self):
        #discount_r = self.discount_rewards(self.rewards)
        #policy,values = self.actor_critic_network(self.states)

        #advantages = discount_r - values
        R = self.discount_rewards(self.rewards)
        R = torch.FloatTensor(R)
        states = torch.FloatTensor(self.states)
        actions = torch.LongTensor(self.actions)
        rewards = torch.FloatTensor(self.rewards)
        next_states = torch.FloatTensor(self.next_states)
        dones = torch.FloatTensor(self.terminals)
        ones = torch.ones_like(dones)
        

        # Compute advantages
        _, next_values = self.actor_critic_network(next_states)
        advantages = R - self.actor_critic_network.critic(states).detach().squeeze()

        # Compute critic loss
        loss_c = torch.nn.MSELoss()
        critic_loss = loss_c(R,self.actor_critic_network.critic(states).squeeze())#, rewards + self.discount * next_values.detach().squeeze() * (ones-dones))

        # Compute actor loss
        policy, _ = self.actor_critic_network(states)
        selected_probs = policy.gather(1, actions.unsqueeze(1))
        actor_loss = -(torch.log(selected_probs) * advantages.detach()).mean()

        self.actor_optimizer.zero_grad()
        self.critic_optimizer.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        self.actor_optimizer.step()
        self.critic_optimizer.step()
        #print("Actor loss",actor_loss.detach().numpy())
        #+print("Critic loss",critic_loss.detach().numpy())
        # Total loss
        total_loss = actor_loss + critic_loss

        return total_loss.detach().numpy()



    def agent_message(self, message):
        if message == "get_sum_reward":
            return self.sum_rewards
        else:
            raise Exception("Unrecognized Message!")

    def get_loss(self):
        return np.average(np.array(self.loss))

In [7]:
import torch
import matplotlib.pyplot as plt
import numpy as np


class Utilities:
    def load_model(self,model,filename="model.weights"):
        if(model.name == "DQN"):
            model.target_network.load_state_dict(torch.load(filename))
            model.q_network.load_state_dict(torch.load(filename))
        elif(model.name=="ActorCritic"):
            model.actor_critic_network.load_state_dict(torch.load(filename))
        else:
            NotImplementedError()


    def save_model(self,model,filename="model.weights"):
        if(model.name == "DQN"):
            torch.save(model.q_network.state_dict(),filename)
        elif(model.name=="ActorCritic"):
            torch.save(model.actor_critic_network.state_dict(),filename)


    def test_agent(self,agent,test_env):
        pass

    def visualize_values(self,agent,env,runs=1):
        for i in range(runs):
            state,_=env.reset()
            done = False
            n_step = 0
            values = []
            while(done!=True):
                action_values = agent.q_network.get_action_values(state)
                values.append(action_values)
                action = np.argmax(action_values)
                state,reward,terminated,truncated,info=env.step(action)
                done = terminated or truncated
            values = np.array(values)
            values = values.reshape(values.shape[0],agent.num_actions)
            print(values.shape)
            plt.plot(values)
        plt.show()

In [8]:
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
#import gymnasium as gym
import numpy as np
#from DQNAgent import DQNAgent
import matplotlib.pyplot as plt
import random
import os

class RL:
    def __init__(self) -> None:
        self.name = "ButaChanRL"
        self.mean_episode_length = 0
        self.mean_episode_rew = 0
        self.mean_loss= 0
        self.step = 0
        self.output_step = 1000
        self.epsiode_rewards = []
        self.episode_lens = []
        self.loss = []
        self.utils = Utilities()
        self.model_dir = "./models/"
        self.num_episodes = 0
        self.average_over = 20

    def set_output_step(self,output_step):
        self.output_step = output_step

    def set_seed(self,seed=1):
        random.seed(seed)
        np.random.seed(seed)

    def set_model_dir(self,name):
        self.model_dir = name

    def create_model_dir(self):
        if not os.path.exists(self.model_dir):
            os.makedirs(self.model_dir)

    def visualize_values(self,agent,env,runs=1):
        self.utils.visualize_values(agent,env,runs)
     
    def plot_live(self,data,n_mean=20,plot_start=20):
        plt.ion()
        plt.figure(1)
        plot_data = torch.tensor(data, dtype=torch.float)
        plt.clf()
        plt.title('Training...')
        plt.xlabel('Episode')
        plt.ylabel('Episode Reward')
        plt.plot(plot_data.numpy(),"o")
        # Take 100 episode averages and plot them too
        if len(plot_data ) >= plot_start:
            means = plot_data .unfold(0, n_mean, 1).mean(1).view(-1)
            means = torch.cat((torch.zeros(n_mean), means))
            plt.plot(means.numpy())
        plt.pause(0.1)  # pause a bit so that plots are updated

    def plot_validate(self,data,test_data):
        plt.ion()
        plt.figure(1)
        plot_data = torch.tensor(data, dtype=torch.float)
        test_data = torch.tensor(test_data,dtype=torch.float32)
        plt.clf()
        plt.title('Training...')
        plt.xlabel('Episode')
        plt.ylabel('Episode Reward')
        plt.plot(plot_data.numpy(),"o")
        plt.plot(test_data.numpy(),"rx")
        plt.pause(0.1)  # pause a bit so that plots are updated

    def episode_summarize(self,episode,episode_reward):
        print(f"Episode: {episode}, Reward: {episode_reward}")

    def summarize(self):
        self.mean_episode_length = 0
        self.mean_episode_rew = 0
        if(len(self.episode_lens)>0):
            if(len(self.episode_lens)>self.average_over):
                self.mean_episode_length = np.average(self.episode_lens[-self.average_over:-1])
                self.mean_episode_rew = np.average(self.epsiode_rewards[-self.average_over:-1])
            else:
                self.mean_episode_length = np.average(self.episode_lens)
                self.mean_episode_rew = np.average(self.epsiode_rewards)
        self.mean_loss = 0
        if(len(self.loss)>0):
            self.mean_loss = np.average(self.loss)
        print(f"Step:{self.step}, Episode:{self.num_episodes} Mean_Epi_Len: {self.mean_episode_length:5.2f},Mean_Epi_Rew {self.mean_episode_rew:5.2f}, Loss: {self.mean_loss:5.2f}")

    def learn(self,agent,env,agent_parameters,NSTEPS=10000,visualize=False,save_best_weights=False):
        epsiode = 1
        
        
        # prepare agent
        agent.agent_init(agent_parameters)
        state,info= env.reset() 
        # choose initial action based on agent's results
        action = agent.agent_start(state)
        done = False
        epsiode_reward = 0
        episode_len = 0
        
        for i in tqdm(range(1,NSTEPS+1)):
            self.step = i
            state,reward,terminated,truncated,info=env.step(action)
            epsiode_reward += reward
            done = terminated or truncated
            if(self.step%self.output_step==0):
                self.summarize()
                if(visualize):
                    if(len(self.epsiode_rewards)>0):
                        self.plot_live(self.epsiode_rewards)
            if(done):
                loss = agent.agent_end(reward)
                self.loss.append(loss)
                if(save_best_weights):
                    self.create_model_dir()
                    if(len(self.epsiode_rewards)==0):
                        model_name = self.model_dir+f"model_{self.step}"
                        self.utils.save_model(agent,model_name)
                    else:
                        if(epsiode_reward>max(self.epsiode_rewards)):
                            model_name = self.model_dir+f"model_{self.step}"
                            self.utils.save_model(agent,model_name)
                self.epsiode_rewards.append(epsiode_reward)
                self.episode_lens.append(episode_len)
                epsiode += 1
                self.num_episodes += 1
                # restart next episode
                state,_= env.reset() 
                action = agent.agent_start(state)
                done = False
                epsiode_reward = 0
                episode_len = 0
            else:
                action = agent.agent_step(reward,state)
                episode_len+=1
        return agent

    def evaluate(self,agent,env,n_episodes=10,seed=1,visualize=False,eval_espilon=0.001):
        epsiode_rewards = []
        for episode in range(1,n_episodes+1):
            state,info = env.reset()
            action = agent.greedy_policy(state,eval_espilon)
            done = False
            epsiode_reward = 0
            episode_len = 0
            while not done:
                state,reward,terminated,truncated,info=env.step(action)
                epsiode_reward += reward
                done = terminated or truncated
                action = agent.greedy_policy(state,eval_espilon)
                episode_len += 1
            epsiode_rewards.append(epsiode_reward)
            self.episode_summarize(episode,epsiode_reward)
            #if(visualize):
            #    env.summarize()
        mean_rew = np.average(epsiode_rewards)
        std_rew = np.std(epsiode_rewards)
        return (mean_rew,std_rew) 

In [12]:
import gymnasium as gym

env = gym.make("MountainCarContinuous-v0") 
env.observation_space

Box([-1.2  -0.07], [0.6  0.07], (2,), float32)

In [19]:
env.action_space.shape[0]

1

In [20]:

import torch

def run():
    env = gym.make("MountainCarContinuous-v0") #TradingEnv(data_file="usdjpy.csv",start_tick=100,end_tick=500,window_length=9,debug=False,pos=False,tech=False)
    #test_env = TradingEnv(data_file="usdjpy.csv",start_tick=10000,end_tick=11000,window_length=9,debug=False,pos=False,tech=False)
    
    
    n_state = env.observation_space.shape[0]
    n_actions = env.action_space.shape[0]

    agent_parameters = {
    'network_config': {
        'state_dim': n_state,
        'num_hidden_units': 128,
        'num_actions': n_actions,
        "network_type":"dqn"
    },
    'replay_buffer_size': 1_000_000,
    'minibatch_sz': 32,
    'observation_size':n_state,
    'num_replay_updates_per_step': 1,
    "step_size": 3e-4,
    'gamma': 0.99,
    'epsilon': 1,
    'update_freq':500,
    'warmup_steps':500,
    'double_dqn':True
    }
    #agent = ActorCriticAgent() #DQNAgent()
    agent = A2CAgent()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    rl = RL()
    #rl.set_seed(1)
    trained_agent = rl.learn(agent,env,agent_parameters,NSTEPS=50_000,visualize=True,save_best_weights=False)
    mean,std=rl.evaluate(trained_agent,env,n_episodes=5,visualize=True)
    print(f"Mean reward: {mean}, Standard deviation: {std}")

In [21]:
run()

TypeError: len() of unsized object

In [2]:
import gymnasium as gym
from matplotlib import pyplot as plt
import numpy as np
env = gym.make('CartPole-v0', render_mode="rgb_array_list")

/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


In [3]:
import numpy as np
import random 
from collections import namedtuple, deque 

##Importing the model (function approximator for Q-table)

import torch
import torch.nn.functional as F
import torch.optim as optim

BUFFER_SIZE = int(1e5)  #replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate
UPDATE_EVERY = 4        # how often to update the network

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Agent():
    """Interacts with and learns form environment."""
    
    def __init__(self, state_size, action_size, seed):
        """Initialize an Agent object.
        
        Params
        =======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """
        
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)
        
        
        #Q- Network
        self.qnetwork_local = QNetwork(state_size, action_size, seed).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size, seed).to(device)
        
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(),lr=LR)
        
        # Replay memory 
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE,BATCH_SIZE,seed)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
        
    def step(self, state, action, reward, next_step, done):
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_step, done)

        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step+1)% UPDATE_EVERY
        if self.t_step == 0:
            # If enough samples are available in memory, get radom subset and learn

            if len(self.memory)>BATCH_SIZE:
                experience = self.memory.sample()
                self.learn(experience, GAMMA)
    def act(self, state, eps = 0):
        """Returns action for given state as per current policy
        Params
        =======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        #Epsilon -greedy action selction
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))
            
    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.
        Params
        =======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples
            gamma (float): discount factor
        """
        states, actions, rewards, next_state, dones = experiences
        ## TODO: compute and minimize the loss
        criterion = torch.nn.MSELoss()
        # Local model is one which we need to train so it's in training mode
        self.qnetwork_local.train()
        # Target model is one with which we need to get our target so it's in evaluation mode
        # So that when we do a forward pass with target model it does not calculate gradient.
        # We will update target model weights with soft_update function
        self.qnetwork_target.eval()
        #shape of output from the model (batch_size,action_dim) = (64,4)
        predicted_targets = self.qnetwork_local(states).gather(1,actions)
    
        with torch.no_grad():
            labels_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)

        # .detach() ->  Returns a new Tensor, detached from the current graph.
        labels = rewards + (gamma* labels_next*(1-dones))
        
        loss = criterion(predicted_targets,labels).to(device)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(self.qnetwork_local,self.qnetwork_target,TAU)
            
    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        Params
        =======
            local model (PyTorch model): weights will be copied from
            target model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter
        """
        for target_param, local_param in zip(target_model.parameters(),
                                           local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1-tau)*target_param.data)
            
class ReplayBuffer:
    """Fixed -size buffe to store experience tuples."""
    
    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.experiences = namedtuple("Experience", field_names=["state",
                                                               "action",
                                                               "reward",
                                                               "next_state",
                                                               "done"])
        self.seed = random.seed(seed)
        
    def add(self,state, action, reward, next_state,done):
        """Add a new experience to memory."""
        e = self.experiences(state,action,reward,next_state,done)
        self.memory.append(e)
        
    def sample(self):
        """Randomly sample a batch of experiences from memory"""
        experiences = random.sample(self.memory,k=self.batch_size)
        
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        
        return (states,actions,rewards,next_states,dones)
    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

In [11]:
env.observation_space

NameError: name 'env' is not defined

In [4]:
agent = Agent(state_size=8,action_size=4,seed=0)

def dqn(n_episodes= 200, max_t = 1000, eps_start=1.0, eps_end = 0.01,
       eps_decay=0.996):
    """Deep Q-Learning
    
    Params
    ======
        n_episodes (int): maximum number of training epsiodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon 
        eps_decay (float): mutiplicative factor (per episode) for decreasing epsilon
        
    """
    scores = [] # list containing score from each episode
    scores_window = deque(maxlen=100) # last 100 scores
    eps = eps_start
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state,eps)
            next_state, reward, done, info, termenated  = env.step(action)
            agent.step(state,action,reward,next_state,done)
            ## above step decides whether we will train(learn) the network
            ## actor (local_qnetwork) or we will fill the replay buffer
            ## if len replay buffer is equal to the batch size then we will
            ## train the network or otherwise we will add experience tuple in our 
            ## replay buffer.
            state = next_state
            score += reward
            if done:
                break
            scores_window.append(score) ## save the most recent score
            scores.append(score) ## sae the most recent score
            eps = max(eps*eps_decay,eps_end)## decrease the epsilon
            print('\rEpisode {}\tAverage Score {:.2f}'.format(i_episode,np.mean(scores_window)), end="")
            if i_episode %100==0:
                print('\rEpisode {}\tAverage Score {:.2f}'.format(i_episode,np.mean(scores_window)))
                
            if np.mean(scores_window)>=200.0:
                print('\nEnvironment solve in {:d} epsiodes!\tAverage score: {:.2f}'.format(i_episode-100,
                                                                                           np.mean(scores_window)))
                torch.save(agent.qnetwork_local.state_dict(),'checkpoint.pth')
                break
    return scores

scores= dqn()

#plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)),scores)
plt.ylabel('Score')
plt.xlabel('Epsiode #')
plt.show()

TypeError: expected np.ndarray (got tuple)

In [6]:
agent = Agent(state_size=8,action_size=4,seed=0)

def dqn(n_episodes= 200, max_t = 1000, eps_start=1.0, eps_end = 0.01,
       eps_decay=0.996):
    """Deep Q-Learning
    
    Params
    ======
        n_episodes (int): maximum number of training epsiodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon 
        eps_decay (float): mutiplicative factor (per episode) for decreasing epsilon
        
    """
    scores = [] # list containing score from each episode
    scores_window = deque(maxlen=100) # last 100 scores
    eps = eps_start
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state,eps)
            next_state, reward, done, info, termenated = env.step(action)
            agent.step(state,action,reward,next_state,done)
            ## above step decides whether we will train(learn) the network
            ## actor (local_qnetwork) or we will fill the replay buffer
            ## if len replay buffer is equal to the batch size then we will
            ## train the network or otherwise we will add experience tuple in our 
            ## replay buffer.
            state = next_state
            score += reward
            if done:
                break
            scores_window.append(score) ## save the most recent score
            scores.append(score) ## sae the most recent score
            eps = max(eps*eps_decay,eps_end)## decrease the epsilon
            print('\rEpisode {}\tAverage Score {:.2f}'.format(i_episode,np.mean(scores_window)), end="")
            if i_episode %100==0:
                print('\rEpisode {}\tAverage Score {:.2f}'.format(i_episode,np.mean(scores_window)))
                
            if np.mean(scores_window)>=200.0:
                print('\nEnvironment solve in {:d} epsiodes!\tAverage score: {:.2f}'.format(i_episode-100,
                                                                                           np.mean(scores_window)))
                torch.save(agent.qnetwork_local.state_dict(),'checkpoint.pth')
                break
    return scores

scores= dqn()

#plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)),scores)
plt.ylabel('Score')
plt.xlabel('Epsiode #')
plt.show()

TypeError: expected np.ndarray (got tuple)